# Guia de Docker para ML

> Objetivo: entender Docker desde cero y levantar un contenedor con la API de este bloque (Bloque 2).

> Que vas a conseguir: construir una imagen reproducible, ejecutar un contenedor, exponer la API y verificarla desde el navegador o `curl`.

> Requisitos minimos: Docker Desktop instalado y en ejecucion, y el proyecto en la carpeta **Bloque 2 - Docker**.

> Nota: ejecuta los comandos en una terminal (PowerShell o bash) desde el directorio del proyecto.

## 1. Conceptos clave

- **Imagen**: plantilla inmutable con el sistema y dependencias. Se construye a partir de un `Dockerfile`.
- **Contenedor**: instancia en ejecucion de una imagen. Es efimero: si lo borras, los cambios internos se pierden.
- **Dockerfile**: receta paso a paso para construir la imagen (base, dependencias, archivos, comando).
- **Build context**: carpeta que Docker usa para copiar archivos al construir. Solo lo que esta en el contexto puede copiarse.
- **Registry**: repositorio de imagenes (Docker Hub, GHCR, etc.).
- **Puerto expuesto**: puerto dentro del contenedor. Para acceder desde el host debes mapearlo con `-p`.
- **Capa (layer)**: cada instruccion del Dockerfile crea una capa. El orden importa para aprovechar cache.
- **Tag**: etiqueta de una imagen (por ejemplo `modelo-practica1:latest`).
- **Volume**: almacenamiento persistente gestionado por Docker.
- **Bind mount**: carpeta del host montada dentro del contenedor (`-v ruta_host:ruta_contenedor`).
- **CMD/ENTRYPOINT**: comando por defecto que se ejecuta cuando arranca el contenedor.
- **Variable de entorno**: parametros configurables con `-e` en `docker run` o `environment` en compose.

## 2. Instalacion y verificacion

### 2.1 Instalacion en Windows (Docker Desktop)

1. Descarga Docker Desktop: https://www.docker.com/products/docker-desktop/
2. Reinicia Windows si el instalador lo pide.
3. Abre Docker Desktop y espera a que este activo (icono en bandeja).
4. Asegurate de tener habilitado WSL2 (Docker Desktop lo solicita).
5. Verifica en terminal:

```bash
docker --version
docker info
```

Si `docker info` falla:
- Revisa que Docker Desktop este iniciado.
- Revisa que WSL2 este operativo y que no haya actualizaciones pendientes.
- En Windows, a veces ayuda reiniciar Docker Desktop.

## 3. Estructura del proyecto (Bloque 2)

Vamos a dockerizar el modelo de este bloque. Asumimos esta estructura minima:

```
Bloque 2 - Docker/
  app.py
  requirements.txt
  flight_delay_model.pkl
  Dockerfile
```

- `app.py` contiene la API (FastAPI) y el arranque del servidor.
- `requirements.txt` lista las dependencias que se instalaran dentro de la imagen.
- `flight_delay_model.pkl` es el modelo entrenado que la API carga al iniciar.
- `Dockerfile` define como construir la imagen.

Consejo: crea un `.dockerignore` para excluir carpetas como `__pycache__`, `.venv` o datos grandes.

Si los nombres difieren, ajusta las rutas en los pasos siguientes.

## 4. Crear un Dockerfile

Crea un archivo `Dockerfile` dentro de **Bloque 2 - Docker** con este contenido. Cada linea se explica debajo:

```Dockerfile
FROM python:3.11-slim

# Directorio de trabajo dentro del contenedor
WORKDIR /app

# Copiamos dependencias primero para aprovechar cache
COPY requirements.txt /app/requirements.txt

# Instalamos dependencias
RUN pip install --no-cache-dir -r /app/requirements.txt

# Copiamos el codigo y el modelo
COPY . /app

# Puerto comun para APIs (ajusta si tu app usa otro)
EXPOSE 8000

# Comando por defecto (ajusta si usas uvicorn o flask)
CMD ["python", "app.py"]
```

Notas importantes:

- `python:3.11-slim` reduce el tamano de la imagen manteniendo compatibilidad.
- El orden `COPY requirements.txt` + `RUN pip install` permite cachear dependencias.
- `COPY . /app` debe incluir el modelo `flight_delay_model.pkl`.
- `EXPOSE 8000` es informativo; el acceso real se habilita con `-p` en `docker run`.
- Si hay archivos innecesarios, usa `.dockerignore` para que no se copien al build context.
- Si tu API usa uvicorn directo, puedes cambiar el `CMD` a la linea indicada en Ajustes comunes.

### 4.2 .dockerignore (que excluir y por que)

Crea un archivo `.dockerignore` para reducir el build context y acelerar builds:

```
__pycache__/
*.pyc
*.pyo
.venv/
.env
*.ipynb
.git/
data/
models_backup/
````

Esto evita copiar archivos grandes o sensibles y mejora el cache.

### 4.1 Dockerfile avanzado (multi-stage y usuarios)

Multi-stage reduce tamano final separando build y runtime:

```Dockerfile
FROM python:3.11-slim AS base
WORKDIR /app
COPY requirements.txt /app/requirements.txt
RUN pip install --no-cache-dir -r /app/requirements.txt

FROM python:3.11-slim
WORKDIR /app
COPY --from=base /usr/local /usr/local
COPY . /app
EXPOSE 8000
CMD ["python", "app.py"]
```

Buenas practicas basicas:
- Crea un usuario no root si tu app lo permite.
- Evita copiar archivos innecesarios (usa `.dockerignore`).
- Fija versiones de dependencias para builds reproducibles.

## 5. Construir la imagen

Desde la carpeta **Bloque 2 - Docker** ejecuta:

```bash
docker build -t modelo-practica1:latest .
```

- `-t` asigna un nombre y etiqueta a la imagen.
- El `.` indica que el build context es la carpeta actual.
- Si editas dependencias, Docker reconstruira capas y puede tardar mas.
- Para ver la imagen creada: `docker images`.
- Si necesitas reconstruir desde cero: `docker build --no-cache -t modelo-practica1:latest .`.

### 5.1 Opciones avanzadas de build

Flags comunes y cuando usarlos:
- `-f Dockerfile.dev` para usar otro Dockerfile.
- `--no-cache` para reconstruir desde cero.
- `--pull` para descargar la base mas reciente.
- `--build-arg KEY=VAL` para pasar argumentos de build.
- `--target etapa` si usas multi-stage.
- `--platform linux/amd64` si necesitas otra arquitectura.
- `--progress=plain` para ver logs completos del build.

Ejemplos:

```bash
docker build -f Dockerfile -t modelo-practica1:latest .
docker build --no-cache -t modelo-practica1:latest .
docker build --build-arg ENV=prod -t modelo-practica1:latest .
docker build --platform linux/amd64 -t modelo-practica1:latest .
```

### 5.2 Optimizacion de imagen y cache

Ideas practicas:
- Ordena el Dockerfile para maximizar cache (dependencias primero).
- Evita `COPY .` antes de instalar dependencias.
- Usa `--no-cache-dir` en pip para no guardar cache.
- Revisa el tamano con `docker images` y `docker history`.

Ejemplo de diagnostico:

```bash
docker images
docker history modelo-practica1:latest
```

## 6. Ejecutar el contenedor

Lanza el contenedor mapeando el puerto:

```bash
docker run --rm -p 8000:8000 --name modelo-p1 modelo-practica1:latest
```

- `--rm` borra el contenedor al detenerse (no la imagen).
- `-p 8000:8000` mapea puerto host:contenedor.
- `--name` asigna un nombre facil de recordar.

Opcionales utiles:
- `-d` para ejecutarlo en segundo plano.
- `-e ENV=dev` para variables de entorno.
- `-v ruta_host:/app/modelos` para montar carpetas del host.

Si tu `app.py` expone otro puerto, cambia `8000:8000` por el correcto.
Si quieres revisar la salida, usa `docker logs modelo-p1`.
Para detenerlo: `docker stop modelo-p1`.

### 6.1 Opciones avanzadas de run

Flags utiles para controlar el contenedor:
- `-p 127.0.0.1:8000:8000` para limitar a localhost.
- `-e ENV=prod` o `--env-file .env` para variables de entorno.
- `-v ./models:/app/models` para montar datos del host.
- `--restart unless-stopped` para reinicio automatico.
- `--cpus 1.0 -m 512m` para limitar recursos.
- `--network nombre_red` para conectar con otros servicios.

Ejemplos:

```bash
docker run -d -p 8000:8000 --name modelo-p1 modelo-practica1:latest
docker run --rm -p 8000:8000 --env-file .env modelo-practica1:latest
docker run --rm -p 8001:8000 -v ./models:/app/models modelo-practica1:latest
```

Inspeccion y gestion:
- Ver puertos: `docker port modelo-p1`
- Entrar al contenedor: `docker exec -it modelo-p1 /bin/sh`
- Parar y borrar: `docker stop modelo-p1` y `docker rm modelo-p1`

### 6.2 Volumenes y persistencia (bind vs volume)

Dos formas de persistir datos:
- **Bind mount**: carpeta del host. Rapido para desarrollo.
- **Volume**: gestionado por Docker. Mejor para produccion.

Ejemplos:

```bash
docker run --rm -v ./models:/app/models modelo-practica1:latest
docker volume create modelos
docker run --rm -v modelos:/app/models modelo-practica1:latest
```

Si montas un volumen sobre una ruta con archivos, esa ruta se sobreescribe por el contenido del volumen.

## 7. Probar la API

Si es una API web, prueba con `curl` o navegador:

```bash
curl http://localhost:8000/
```

En FastAPI, tambien tienes documentacion interactiva en:
- `http://localhost:8000/docs` (Swagger UI)
- `http://localhost:8000/redoc`

Ejemplo de prediccion individual:

```bash
curl -X POST http://localhost:8000/predict -H "Content-Type: application/json" -d '{"flight_id": "ABC123", "distance": 1200, "bad_weather": false}'
```

Ejemplo por lotes:

```bash
curl -X POST http://localhost:8000/predict-batch -H "Content-Type: application/json" -d '[{"flight_id": "A1", "distance": 900, "bad_weather": true}, {"flight_id": "B2", "distance": 1500, "bad_weather": false}]'
```

Endpoints utiles para comprobar que todo esta vivo:
- `GET /info` para ver metadatos del servicio.
- `GET /metrics` para ver contador de predicciones y uptime.

## 8. Ajustes comunes

- **uvicorn / FastAPI**: cambia el CMD por:

```Dockerfile
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
```

- **Flask**: usa `flask run --host=0.0.0.0 --port=8000` o `python app.py` segun tu app.
- **El contenedor se para al arrancar**: revisa `docker logs <nombre>` para ver errores o si la app termina.
- **No responde en localhost**: confirma que la app escucha en `0.0.0.0` y que el puerto mapeado coincide.
- **Problemas con el modelo**: verifica que `flight_delay_model.pkl` esta dentro del contenedor (`COPY . /app`).
- **Warnings de sklearn**: si hay versiones distintas, actualiza `requirements.txt` o reentrena el modelo.
- **Puertos ocupados**: si 8000 esta ocupado, cambia a `-p 8001:8000` y entra por `http://localhost:8001`.

### 8.1 Debug y troubleshooting rapido

Comandos utiles:
- Ver contenedores: `docker ps -a`
- Ver logs: `docker logs <nombre>`
- Ver detalles: `docker inspect <nombre>`
- Entrar al contenedor: `docker exec -it <nombre> /bin/sh`
- Probar desde dentro: `python -c "import joblib"`

Si la API no responde:
- Verifica que el contenedor siga en `docker ps`.
- Revisa logs para errores al arrancar.
- Confirma puerto y host `0.0.0.0`.

### 8.2 Seguridad basica

Buenas practicas simples:
- No ejecutes como root si no es necesario.
- Evita guardar secretos en la imagen (usa variables de entorno).
- Usa versiones fijadas en `requirements.txt`.
- Mantiene la base actualizada con `--pull` en el build.
- Revisa que el modelo y dependencias vengan de fuentes confiables.

## 9. Siguiente paso: docker-compose (opcional)

Si necesitas variables de entorno, volumenes o varios servicios, podemos crear un `docker-compose.yml`.
Compose es una herramienta para definir y ejecutar aplicaciones multi-contenedor con un archivo YAML.
Te permite versionar la configuracion y levantar todo con un solo comando.

Ejemplo minimo para esta API:

```yaml
services:
  api:
    build: .
    ports:
      - "8000:8000"
    environment:
      - ENV=dev
```

Se ejecuta con:

```bash
docker compose up --build
```

Comandos utiles:
- `docker compose up -d` para segundo plano.
- `docker compose logs -f` para ver logs en tiempo real.
- `docker compose ps` para ver estado de servicios.
- `docker compose down` para parar y eliminar contenedores de ese stack.
- `docker compose down -v` para borrar volumenes creados.

Diferencias con `docker run`:
- Compose gestiona varios servicios a la vez (api, db, etc.).
- Puedes versionar la configuracion y compartirla con el equipo.
- Permite definir redes y volumenes declarativamente.

Campos comunes en `docker-compose.yml`:
- `build` (contexto, dockerfile, args).
- `image` (nombre de la imagen resultante).
- `ports` (mapeo host:contenedor).
- `environment` / `env_file` (variables).
- `volumes` (persistencia y mounts).
- `depends_on` (orden de arranque).
- `healthcheck` (estado del servicio).

### 9.1 docker-compose con nombre de imagen y red

Tambien puedes fijar un nombre de imagen y una red personalizada:

```yaml
services:
  api:
    build: .
    image: modelo-practica1:latest
    ports:
      - "8000:8000"
networks:
  default:
    name: red-ml
```

Esto facilita reutilizar la imagen y ver los contenedores dentro de una red dedicada.

### 9.2 docker-compose con volumenes y reinicio

Ejemplo mas completo con volumen y politica de reinicio:

```yaml
services:
  api:
    build: .
    ports:
      - "8000:8000"
    environment:
      - ENV=dev
    volumes:
      - ./models:/app/models
    restart: unless-stopped
```

- `volumes` permite cargar modelos o datos sin reconstruir la imagen.
- `restart: unless-stopped` reinicia el contenedor si se cae.
- Si usas `volumes`, asegurate de que tu app lea desde esa ruta.

Si necesitas que el contenedor espere a otro servicio (por ejemplo una base de datos), puedes usar `depends_on`.

### 9.3 docker-compose con env_file y healthcheck

Ejemplo con variables externas y healthcheck usando Python (sin curl):

```yaml
services:
  api:
    build:
      context: .
      args:
        - ENV=prod
    ports:
      - "8000:8000"
    env_file:
      - .env
    healthcheck:
      test: ["CMD", "python", "-c", "import urllib.request; urllib.request.urlopen('http://localhost:8000/info')"]
      interval: 30s
      timeout: 5s
      retries: 3
```

- `env_file` carga variables desde un archivo `.env`.
- `healthcheck` marca el servicio como healthy cuando responde correctamente.

## 10. Ejercicios sugeridos

### Ejercicio 1: Cambiar el CMD

Objetivo: arrancar la API con `uvicorn` y comprobar el endpoint de docs.

Tarea:
- Cambia el `CMD` del Dockerfile para usar `uvicorn` directamente.
- Reconstruye y ejecuta el contenedor.

Comprobacion:
- Abre `http://localhost:8000/docs` y verifica que carga.

### Ejercicio 2: Volumen para el modelo

Objetivo: cargar el modelo desde el host sin reconstruir la imagen.

Tarea:
- Monta el archivo `flight_delay_model.pkl` o una carpeta `models/` en `/app`.

Comprobacion:
- La API debe arrancar y seguir prediciendo con el modelo montado.

### Ejercicio 3: Imagen versionada

Objetivo: crear una imagen con etiqueta `v1` y comprobarla.

Tarea:
- Construye con `docker build -t modelo-practica1:v1 .` o etiqueta desde `latest`.

Comprobacion:
- `docker images` debe mostrar `modelo-practica1` con `v1`.

### Ejercicio 4: Variable de entorno en la API

Objetivo: leer `ENV` desde el contenedor y mostrarlo en `/info`.

Tarea:
- Añade `ENV = os.getenv("ENV", "dev")` en `app.py`.
- Incluye `environment` en la respuesta de `/info`.
- Ejecuta con `-e ENV=prod`.

Comprobacion:
- `GET /info` debe devolver `environment` con `prod`.

### Ejercicio 5: Cambiar version de Python y comparar

Objetivo: comparar tamano y tiempo de build con otra base.

Tarea:
- Cambia `FROM python:3.11-slim` por `FROM python:3.10-slim`.
- Construye una imagen nueva con tag diferente (por ejemplo `py310`).

Comprobacion:
- Compara tamano con `docker images` y capas con `docker history`.

---------
## 11. Soluciones de los ejercicios

Cada solucion esta en su propia celda con pasos y comandos.

### 11.1 Cambiar el CMD y probar otra forma de arrancar

Objetivo: arrancar FastAPI con `uvicorn` sin depender de `python app.py`.

Opcion A (editar `Dockerfile`):

```Dockerfile
# Reemplaza el CMD actual por uvicorn directo
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
```

Reconstruye y ejecuta:

```bash
docker build -t modelo-practica1:latest .
docker run --rm -p 8000:8000 --name modelo-p1 modelo-practica1:latest
```

Opcion B (sin tocar Dockerfile, sobreescribe el comando en `docker run`):

```bash
docker run --rm -p 8000:8000 modelo-practica1:latest uvicorn app:app --host 0.0.0.0 --port 8000
```

Verifica en `http://localhost:8000/docs`.

### 11.2 Montar un volumen para cargar el modelo del host

Objetivo: no reconstruir la imagen cada vez que cambias el modelo.

Opcion A (montar el archivo del modelo):

```bash
docker run --rm -p 8000:8000 -v ./flight_delay_model.pkl:/app/flight_delay_model.pkl modelo-practica1:latest
```

Opcion B (montar una carpeta `models/`):

```bash
docker run --rm -p 8000:8000 -v ./models:/app/models modelo-practica1:latest
```

Si usas la carpeta, la app debe leer desde `/app/models/flight_delay_model.pkl`.
Tip: si el contenedor no arranca, revisa permisos de la ruta en Windows y usa rutas absolutas si es necesario.

### 11.3 Crear una imagen versionada

Objetivo: mantener varias versiones publicables de la imagen.

Construye con etiqueta `v1`:

```bash
docker build -t modelo-practica1:v1 .
```

Si ya la construiste como `latest`, etiqueta sin reconstruir:

```bash
docker tag modelo-practica1:latest modelo-practica1:v1
```

Comprueba las etiquetas:

```bash
docker images | findstr modelo-practica1
```

### 11.4 Agregar una variable de entorno y leerla en `app.py`

Objetivo: configurar el comportamiento sin tocar el codigo.

1) En `app.py` (cerca de la inicializacion):

```python
import os
ENV = os.getenv("ENV", "dev")
```

2) Exponlo en el endpoint `/info`:

```python
"environment": ENV,
```

3) Ejecuta con variable:

```bash
docker run --rm -p 8000:8000 -e ENV=prod modelo-practica1:latest
```

Comprueba el valor en `http://localhost:8000/info`.

### 11.5 Construir con otra version de Python y comparar

Objetivo: comparar tamano y tiempo de build con otra base.

1) Cambia la linea del Dockerfile:

```Dockerfile
FROM python:3.10-slim
```

2) Reconstruye:

```bash
docker build -t modelo-practica1:py310 .
```

3) Compara tamanos y capas:

```bash
docker images | findstr modelo-practica1
docker history modelo-practica1:py310
```

Tip: para comparar tiempos, usa `--no-cache` en ambos builds.